In [ ]:
# default_exp data.language_modeling


In [ ]:
# all_slow


In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.language_modeling

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for causal and masked language modeling tasks. This includes things like training BERT from scratch or fine-tuning a particular pre-trained LM on your own corpus.

**This is currently a work in progress** - You've been warned : )

In [ ]:
# export
import os, random
from abc import ABC, abstractmethod
from enum import Enum

from datasets import Dataset
from fastcore.all import *
from fastai.imports import *
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForCausalLM, AutoModelForMaskedLM, logging, PretrainedConfig, PreTrainedTokenizerBase, PreTrainedModel

from blurr.utils import BLURR
from blurr.data.core import TextInput, BatchTokenizeTransform, Preprocessor, first_blurr_tfm

logging.set_verbosity_error()


In [ ]:
# hide_input
import pdb

from fastai.data.block import DataBlock, ColReader, ColSplitter
from fastai.data.core import DataLoader, DataLoaders, TfmdDL
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import *
from fastcore.test import *
from nbdev.showdoc import show_doc

from blurr.utils import print_versions
from blurr.data.core import TextBlock

os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("What we're running with at the time this documentation was generated:")
print_versions("torch fastai transformers")


What we're running with at the time this documentation was generated:
torch: 1.10.1+cu111
fastai: 2.5.3
transformers: 4.16.2


In [ ]:
# hide
# cuda
torch.cuda.set_device(1)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #1: GeForce GTX 1080 Ti


## Setup

For this example, we'll use the `WIKITEXT_TINY` dataset available from fastai.  In addition to using the `Datasets` library from Hugging Face, fastai provides a lot of smaller datasets that are really useful when experimenting and/or in the early development of your training/validation/inference coding.

In [ ]:
wiki_path = untar_data(URLs.WIKITEXT_TINY)
wiki_path.ls()


(#2) [Path('/home/wgilliam/.fastai/data/wikitext-2/train.csv'),Path('/home/wgilliam/.fastai/data/wikitext-2/test.csv')]

In [ ]:
train_df = pd.read_csv(wiki_path / "train.csv", header=None)
valid_df = pd.read_csv(wiki_path / "test.csv", header=None)

print(len(train_df), len(valid_df))
train_df.head()


615 47


,0
0,"\n = 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club , a professional football club based in York , North Yorkshire , England . Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two . The season ran from 1 July 2013 to 30 June 2014 . \n Nigel Worthington , starting his first full season as York manager , made eight permanent summer signings . By the turn of the year York were only above the relegation z..."
1,"\n = Big Boy ( song ) = \n \n "" Big Boy "" <unk> "" I 'm A Big Boy Now "" was the first single ever recorded by the Jackson 5 , which was released by Steeltown Records in January 1968 . The group played instruments on many of their Steeltown compositions , including "" Big Boy "" . The song was neither a critical nor commercial success , but the Jackson family were delighted with the outcome nonetheless . \n The Jackson 5 would release a second single with Steeltown Records before moving to Motown Records . The group 's recordings at Steeltown Records were thought to be lost , but they were re..."
2,"\n = The Remix ( Lady Gaga album ) = \n \n The Remix is a remix album by American recording artist Lady Gaga . Released in Japan on March 3 , 2010 , it contains remixes of the songs from her first studio album , The Fame ( 2008 ) , and her third extended play , The Fame Monster ( 2009 ) . A revised version of the track list was prepared for release in additional markets , beginning with Mexico on May 3 , 2010 . A number of recording artists have produced the songs , including Pet Shop Boys , Passion Pit and The Sound of Arrows . The remixed versions feature both uptempo and <unk> composit..."
3,"\n = New Year 's Eve ( Up All Night ) = \n \n "" New Year 's Eve "" is the twelfth episode of the first season of the American comedy television series Up All Night . The episode originally aired on NBC in the United States on January 12 , 2012 . It was written by Erica <unk> and was directed by Beth McCarthy @-@ Miller . The episode also featured a guest appearance from Jason Lee as Chris and Reagan 's neighbor and Ava 's boyfriend , Kevin . \n During Reagan ( Christina Applegate ) and Chris 's ( Will <unk> ) first New Year 's Eve game night , Reagan 's competitiveness comes out causing Ch..."
4,"\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the fungus are reddish @-@ brown with a whitish fringe and measure up to 2 cm ( 0 @.@ 8 in ) across . They have a short , tapered stalk . Fruitbodies are commonly found on soil where brush has recently been burned , sometimes in great numbers ...."


In [ ]:
train_df["is_valid"] = False
valid_df["is_valid"] = True

df = pd.concat([train_df, valid_df])
df.head()


,0,is_valid
0,"\n = 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club , a professional football club based in York , North Yorkshire , England . Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two . The season ran from 1 July 2013 to 30 June 2014 . \n Nigel Worthington , starting his first full season as York manager , made eight permanent summer signings . By the turn of the year York were only above the relegation z...",False
1,"\n = Big Boy ( song ) = \n \n "" Big Boy "" <unk> "" I 'm A Big Boy Now "" was the first single ever recorded by the Jackson 5 , which was released by Steeltown Records in January 1968 . The group played instruments on many of their Steeltown compositions , including "" Big Boy "" . The song was neither a critical nor commercial success , but the Jackson family were delighted with the outcome nonetheless . \n The Jackson 5 would release a second single with Steeltown Records before moving to Motown Records . The group 's recordings at Steeltown Records were thought to be lost , but they were re...",False
2,"\n = The Remix ( Lady Gaga album ) = \n \n The Remix is a remix album by American recording artist Lady Gaga . Released in Japan on March 3 , 2010 , it contains remixes of the songs from her first studio album , The Fame ( 2008 ) , and her third extended play , The Fame Monster ( 2009 ) . A revised version of the track list was prepared for release in additional markets , beginning with Mexico on May 3 , 2010 . A number of recording artists have produced the songs , including Pet Shop Boys , Passion Pit and The Sound of Arrows . The remixed versions feature both uptempo and <unk> composit...",False
3,"\n = New Year 's Eve ( Up All Night ) = \n \n "" New Year 's Eve "" is the twelfth episode of the first season of the American comedy television series Up All Night . The episode originally aired on NBC in the United States on January 12 , 2012 . It was written by Erica <unk> and was directed by Beth McCarthy @-@ Miller . The episode also featured a guest appearance from Jason Lee as Chris and Reagan 's neighbor and Ava 's boyfriend , Kevin . \n During Reagan ( Christina Applegate ) and Chris 's ( Will <unk> ) first New Year 's Eve game night , Reagan 's competitiveness comes out causing Ch...",False
4,"\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the fungus are reddish @-@ brown with a whitish fringe and measure up to 2 cm ( 0 @.@ 8 in ) across . They have a short , tapered stalk . Fruitbodies are commonly found on soil where brush has recently been burned , sometimes in great numbers ....",False


In [ ]:
model_cls = AutoModelForCausalLM

pretrained_model_name = "gpt2"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

# some tokenizers like gpt and gpt2 do not have a pad token, so we add it here mainly for the purpose
# of setting the "labels" key appropriately (see below)
if hf_tokenizer.pad_token is None:
    hf_tokenizer.pad_token = "[PAD]"

hf_tokenizer.pad_token, hf_tokenizer.pad_token_id


Using pad_token, but it is not set yet.


('[PAD]', 50256)

### `LMPreprocessor`

Starting with version 2.0, BLURR provides a language preprocessing class that can be used to preprocess DataFrames or Hugging Face Datasets for both causal and masked language modeling tasks.

In [ ]:
# export
class LMPreprocessor(Preprocessor):
    def __init__(
        self,
        # A Hugging Face tokenizer
        hf_tokenizer: PreTrainedTokenizerBase,
        # The number of examples to process at a time
        batch_size: int = 1000,
        # Whether the dataset should be processed for multi-label; if True, will ensure `label_attrs` are
        # converted to a value of either 0 or 1 indiciating the existence of the class in the example
        is_multilabel: bool = False,
        # The unique identifier in the dataset
        id_attr: Optional[str] = None,
        # The attribute holding the text
        text_attr: str = "text",
        # The attribute holding the text_pair
        text_pair_attr: Optional[str] = None,
        # The attribute holding the label(s) of the example
        label_attrs: Union[str, List[str]] = "label",
        # The attribute that should be created if your are processing individual training and validation
        # datasets into a single dataset, and will indicate to which each example is associated
        is_valid_attr: Optional[str] = "is_valid",
        # A list indicating the valid labels for the dataset (optional, defaults to the unique set of labels
        # found in the full dataset)
        label_mapping: Optional[List[str]] = None,
        # Tokenization kwargs that will be applied with calling the tokenizer
        tok_kwargs: dict = {},
    ):
        tok_kwargs = {**tok_kwargs, "return_offsets_mapping": True}
        super().__init__(hf_tokenizer, batch_size, text_attr, text_pair_attr, is_valid_attr, tok_kwargs)

        self.is_multilabel = is_multilabel
        self.id_attr = id_attr
        self.label_attrs = label_attrs
        self.label_mapping = label_mapping

    def process_df(self, training_df: pd.DataFrame, validation_df: Optional[pd.DataFrame] = None):
        df = super().process_df(training_df, validation_df)

        # convert even single "labels" to a list to make things easier
        label_cols = listify(self.label_attrs)

        # if "is_multilabel", convert all targets to an int, 0 or 1, rounding floats if necessary
        if self.is_multilabel:
            for label_col in label_cols:
                df[label_col] = df[label_col].apply(lambda v: int(bool(max(0, round(v)))))

        # if a "label_mapping" is included, add a "[label_col]_name" field with the label Ids converted to their label names
        if self.label_mapping:
            for label_col in label_cols:
                df[f"{label_col}_name"] = df[label_col].apply(lambda v: self.label_mapping[v])

        # process df in mini-batches
        final_df = pd.DataFrame()
        for g, batch_df in df.groupby(np.arange(len(df)) // self.batch_size):
            final_df = final_df.append(self._process_df_batch(batch_df))

        final_df.reset_index(drop=True, inplace=True)
        return final_df

    def process_hf_dataset(self, training_ds: Dataset, validation_ds: Optional[Dataset] = None):
        ds = super().process_hf_dataset(training_ds, validation_ds)
        return Dataset.from_pandas(self.process_df(pd.DataFrame(ds)))

    # ----- utility methods -----
    def _process_df_batch(self, batch_df):
        batch_df.reset_index(drop=True, inplace=True)

        # grab our inputs
        inputs = self._tokenize_function(batch_df.to_dict(orient="list"))

        for txt_seq_idx, txt_attr in enumerate([self.text_attr, self.text_pair_attr]):
            if txt_attr is None:
                break

            char_idxs = []
            for idx, offset_mapping in enumerate(inputs["offset_mapping"]):
                text_offsets = [offset_mapping[i] for i, seq_id in enumerate(inputs.sequence_ids(idx)) if seq_id == txt_seq_idx]
                char_idxs.append([min(text_offsets)[0], max(text_offsets)[1]])

            batch_df = pd.concat(
                [batch_df, pd.DataFrame(char_idxs, columns=[f"{txt_attr}_start_char_idx", f"{txt_attr}_end_char_idx"])], axis=1
            )
            batch_df.insert(
                0,
                f"proc_{txt_attr}",
                batch_df.apply(lambda r: r[txt_attr][r[f"{txt_attr}_start_char_idx"] : r[f"{txt_attr}_end_char_idx"] + 1], axis=1),
            )

        return batch_df


#### Using a `DataFrame`

In [ ]:
# preprocessor = LMPreprocessor(hf_tokenizer, tok_kwargs={"max_length": 24})
# proc_df = preprocessor.process_df(df)
# proc_df.columns, len(proc_df)
# proc_df.head(2)


#### Using a Hugging Face `Dataset`

## LM Strategies

#### `LMType`

BLURR supports both causal and masked language model tasks. To indicate what kind you are training, we introduce this enum

In [ ]:
# export
class LMType(Enum):
    CAUSAL = 1
    MASKED = 2


### `BaseLMStrategy` and implementations

Here we include a `BaseLMStrategy` abstract class and several different strategies for building your inputs and targets for causal and masked language modeling tasks.  With CLMs, the objective is to simply predict the next token, but with MLMs, a variety of masking strategies may be used (e.g., mask random tokens, mask random words, mask spans, etc...).  A `BertMLMStrategy` is introduced below that follows the "mask random tokens" strategy used in the BERT paper, but users can create their own `BaseLMStrategy` subclass to support any masking strategy they desire.

In [ ]:
# export
class BaseLMStrategy(ABC):
    """ABC for various language modeling strategies (e.g., causal, BertMLM, WholeWordMLM, etc...)"""

    def __init__(self, hf_tokenizer, ignore_token_id=CrossEntropyLossFlat().ignore_index):
        store_attr(["hf_tokenizer", "ignore_token_id"])

    @abstractmethod
    def build_inputs_targets(self, samples):
        pass

    # utility methods
    def _get_random_token_id(self, n):
        return random.sample(list(self.hf_tokenizer.get_vocab().values()), n)

    @classmethod
    @abstractmethod
    def get_lm_type(cls):
        pass


#### `CausalLMStrategy`

For predicting the next token given the prior tokens

In [ ]:
# export
class CausalLMStrategy(BaseLMStrategy):
    """For next token prediction language modeling tasks, we want to use the `CausalLMStrategy` which makes the
    necessary changes in your inputs/targets for causal LMs
    """

    def build_inputs_targets(self, samples):
        updated_samples = []
        for s in samples:
            s[0]["labels"] = s[0]["input_ids"].clone()
            s[0]["labels"][s[0]["labels"] == self.hf_tokenizer.pad_token_id] = self.ignore_token_id
            targ_ids = torch.cat([s[0]["input_ids"][1:], tensor([self.hf_tokenizer.eos_token_id])])

            updated_samples.append((s[0], targ_ids))

        return updated_samples

    @classmethod
    def get_lm_type(cls: LMType):
        return LMType.CAUSAL


#### `BertMLMStrategy`

Follows the masking strategy used in the [BERT paper](https://arxiv.org/abs/1810.04805) for random token masking

In [ ]:
# export
class BertMLMStrategy(BaseLMStrategy):
    """A masked language modeling strategy using the default BERT masking definition."""

    def __init__(self, hf_tokenizer, ignore_token_id=CrossEntropyLossFlat().ignore_index):
        super().__init__(hf_tokenizer, ignore_token_id)

        vocab = hf_tokenizer.get_vocab()
        self.dnm_tok_ids = [
            vocab[tok] for tok in list(hf_tokenizer.special_tokens_map.values()) if vocab[tok] != hf_tokenizer.mask_token_id
        ]

    def build_inputs_targets(self, samples):
        updated_samples = []
        for s in samples:
            # mask the input_ids
            masked_input_ids = s[0]["input_ids"].clone()

            # we want to mask 15% of the non-special tokens(e.g., special tokens inclue [CLS], [SEP], etc...)
            idxs = torch.randperm(len(masked_input_ids))
            total_masked_idxs = int(len(idxs) * 0.15)

            # of the 15% for masking, replace 80% with [MASK] token, 10% with random token, and 10% with correct token
            n_mask_idxs = int(total_masked_idxs * 0.8)
            n_rnd_idxs = int(total_masked_idxs * 0.1)

            # we only want non-special tokens
            mask_idxs = [idx for idx in idxs if masked_input_ids[idx] not in self.dnm_tok_ids][:total_masked_idxs]

            # replace 80% with [MASK]
            if n_mask_idxs > 0 and len(mask_idxs) >= n_mask_idxs:
                masked_input_ids[[mask_idxs[:n_mask_idxs]]] = self.hf_tokenizer.mask_token_id

            # replace 10% with a random token
            if n_rnd_idxs > 0 and len(mask_idxs) >= (n_mask_idxs + n_rnd_idxs):
                rnd_tok_ids = self._get_random_token_id(n_rnd_idxs)
                masked_input_ids[[mask_idxs[n_mask_idxs : (n_mask_idxs + n_rnd_idxs)]]] = tensor(rnd_tok_ids)

            # ignore padding when calculating the loss
            lbls = s[0]["input_ids"].clone()
            lbls[[[idx for idx in idxs if idx not in mask_idxs]]] = self.ignore_token_id

            # update the inputs to use our masked input_ids and labels; set targ_ids = labels (will use when
            # we calculate the loss ourselves)
            s[0]["input_ids"] = masked_input_ids
            s[0]["labels"] = lbls
            targ_ids = lbls.clone()

            updated_samples.append((s[0], targ_ids))

        return updated_samples

    @classmethod
    def get_lm_type(cls: LMType):
        return LMType.MASKED


## Mid-level API

#### `CausalLMTextInput` and `MLMTextInput`

Again, we define a custom classes for the `@typedispatch`ed methods to use so that we can override how both causal and masked language modeling inputs/targets are assembled, as well as, how the data is shown via methods like `show_batch` and `show_results`.

In [ ]:
# export
class CausalLMTextInput(TextInput):
    pass


# export
class MLMTextInput(TextInput):
    pass


#### `LMBatchTokenizeTransform`

Our `LMBatchTokenizeTransform` allows us to update the input's `labels` and our targets appropriately given any language modeling task. 

The `labels` argument allows you to forgo calculating the loss yourself by letting Hugging Face return it for you should you choose to do that. Padding tokens are set to -100 by default (e.g., `CrossEntropyLossFlat().ignore_index`) and prevent cross entropy loss from considering token prediction for tokens it should ... i.e., the padding tokens. For more information on the meaning of this argument, see the [Hugging Face glossary entry for "Labels"](https://huggingface.co/transformers/glossary.html#labels)

In [ ]:
# export
class LMBatchTokenizeTransform(BatchTokenizeTransform):
    def __init__(
        self,
        # The abbreviation/name of your Hugging Face transformer architecture (e.b., bert, bart, etc..)
        hf_arch: str,
        # A specific configuration instance you want to use
        hf_config: PretrainedConfig,
        # A Hugging Face tokenizer
        hf_tokenizer: PreTrainedTokenizerBase,
        # A Hugging Face model
        hf_model: PreTrainedModel,
        # To control whether the "labels" are included in your inputs. If they are, the loss will be calculated in
        # the model's forward function and you can simply use `PreCalculatedLoss` as your `Learner`'s loss function to use it
        include_labels: bool = True,
        # The token ID that should be ignored when calculating the loss
        ignore_token_id: int = CrossEntropyLossFlat().ignore_index,
        # The language modeling strategy (or objective)
        lm_strategy_cls: BaseLMStrategy = CausalLMStrategy,
        # To control the length of the padding/truncation. It can be an integer or None,
        # in which case it will default to the maximum length the model can accept. If the model has no
        # specific maximum input length, truncation/padding to max_length is deactivated.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        max_length: int = None,
        # To control the `padding` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `'do_not_pad'.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        padding: Union[bool, str] = True,
        # To control `truncation` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `do_not_truncate`.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        truncation: Union[bool, str] = True,
        # The `is_split_into_words` argument applied to your `hf_tokenizer` during tokenization. Set this to `True`
        # if your inputs are pre-tokenized (not numericalized)
        is_split_into_words: bool = False,
        # Any other keyword arguments you want included when using your `hf_tokenizer` to tokenize your inputs
        tok_kwargs={},
        # Any keyword arguments you want included when generated text
        # See [How to generate text](https://huggingface.co/blog/how-to-generate)
        text_gen_kwargs={},
        # Keyword arguments to apply to `BatchTokenizeTransform`
        **kwargs
    ):
        super().__init__(
            hf_arch,
            hf_config,
            hf_tokenizer,
            hf_model,
            include_labels=include_labels,
            ignore_token_id=ignore_token_id,
            max_length=max_length,
            padding=padding,
            truncation=truncation,
            is_split_into_words=is_split_into_words,
            tok_kwargs=tok_kwargs.copy(),
            **kwargs
        )

        self.lm_strategy = lm_strategy_cls(hf_tokenizer=hf_tokenizer, ignore_token_id=ignore_token_id)
        self.text_gen_kwargs, self.ignore_token_id = text_gen_kwargs, ignore_token_id

    def encodes(self, samples):
        # because no target is specific in CLM, fastai will duplicate the inputs (which is just the raw text)
        samples = super().encodes(samples)
        if len(samples[0]) == 1:
            return samples

        return self.lm_strategy.build_inputs_targets(samples)


## Examples

### Using the mid-level API

#### Causal LM

##### Step 1: Get your Hugging Face objects.

In [ ]:
model_cls = AutoModelForCausalLM

pretrained_model_name = "gpt2"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

# some tokenizers like gpt and gpt2 do not have a pad token, so we add it here mainly for the purpose
# of setting the "labels" key appropriately (see below)
if hf_tokenizer.pad_token is None:
    hf_tokenizer.pad_token = "[PAD]"

hf_tokenizer.pad_token, hf_tokenizer.pad_token_id


Using pad_token, but it is not set yet.


('[PAD]', 50256)

#####  Step 2: Create your `DataBlock`

In [ ]:
batch_tok_tfm = LMBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, lm_strategy_cls=CausalLMStrategy)

blocks = (TextBlock(batch_tokenize_tfm=batch_tok_tfm, input_return_type=CausalLMTextInput), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader(0), splitter=ColSplitter(col="is_valid"))


##### Step 3: Build your `DataLoaders`

In [ ]:
dls = dblock.dataloaders(df, bs=4)


In [ ]:
b = dls.one_batch()


In [ ]:
b[0]["input_ids"].shape, b[0]["labels"].shape, b[1].shape


(torch.Size([4, 1024]), torch.Size([4, 1024]), torch.Size([4, 1024]))

In [ ]:
explode_types(b)


{tuple: [dict, torch.Tensor]}

In [ ]:
# export
@typedispatch
def show_batch(
    # This typedispatched `show_batch` will be called for `CausalLMTextInput` typed inputs
    x: CausalLMTextInput,
    # Your targets
    y,
    # Your raw inputs/targets
    samples,
    # Your `DataLoaders`. This is required so as to get at the Hugging Face objects for
    # decoding them into something understandable
    dataloaders,
    # Your `show_batch` context
    ctxs=None,
    # The maximum number of items to show
    max_n=6,
    # Any truncation your want applied to your decoded inputs
    trunc_at=None,
    # Any other keyword arguments you want applied to `show_batch`
    **kwargs
):
    # grab our tokenizer and ignore token to decode
    tfm = first_blurr_tfm(dataloaders)
    hf_tokenizer = tfm.hf_tokenizer
    ignore_token_id = tfm.ignore_token_id

    res = L(
        [
            (
                hf_tokenizer.decode(s[0], skip_special_tokens=False)[:trunc_at],
                hf_tokenizer.decode(s[1][s[1] != ignore_token_id], skip_special_tokens=True)[:trunc_at],
            )
            for s in samples
        ]
    )

    display_df(pd.DataFrame(res, columns=["text", "target"])[:max_n])
    return ctxs


In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=500)


,text,target
0,"\n = Bob Dylan = \n \n Bob Dylan ( / <unk> / ; born Robert Allen Zimmerman, May 24, 1941 ) is an American singer @-@ songwriter, artist and writer. He has been influential in popular music and culture for more than five decades. Much of his most celebrated work dates from the 1960s when his songs chronicled social unrest, although Dylan repudiated suggestions from journalists that he was a spokesman for his generation. Nevertheless, early songs such as "" Blowin'in the Wind "" and "" The Times They A","\n = Bob Dylan = \n \n Bob Dylan ( / <unk> / ; born Robert Allen Zimmerman, May 24, 1941 ) is an American singer @-@ songwriter, artist and writer. He has been influential in popular music and culture for more than five decades. Much of his most celebrated work dates from the 1960s when his songs chronicled social unrest, although Dylan repudiated suggestions from journalists that he was a spokesman for his generation. Nevertheless, early songs such as "" Blowin'in the Wind "" and "" The Times They Ar"
1,"\n = Laurence Olivier = \n \n Laurence Kerr Olivier, Baron Olivier, <unk> ( / <unk> <unk> <unk> / ; 22 May 1907 – 11 July 1989 ) was an English actor who, along with his contemporaries Ralph Richardson and John Gielgud, dominated the British stage of the mid @-@ 20th century. He also worked in films throughout his career, playing more than fifty cinema roles. Late in his career, he had considerable success in television roles. \n His family had no theatrical connections, but Olivier's father, a cle","\n = Laurence Olivier = \n \n Laurence Kerr Olivier, Baron Olivier, <unk> ( / <unk> <unk> <unk> / ; 22 May 1907 – 11 July 1989 ) was an English actor who, along with his contemporaries Ralph Richardson and John Gielgud, dominated the British stage of the mid @-@ 20th century. He also worked in films throughout his career, playing more than fifty cinema roles. Late in his career, he had considerable success in television roles. \n His family had no theatrical connections, but Olivier's father, a cler"


#### Masked LM

##### Step 1: Get your Hugging Face objects.

In [ ]:
model_cls = AutoModelForMaskedLM

pretrained_model_name = "bert-base-uncased"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

# some tokenizers like gpt and gpt2 do not have a pad token, so we add it here mainly for the purpose
# of setting the "labels" key appropriately (see below)
if hf_tokenizer.pad_token is None:
    hf_tokenizer.pad_token = "[PAD]"

hf_tokenizer.pad_token, hf_tokenizer.pad_token_id


('[PAD]', 0)

#####  Step 2: Create your `DataBlock`

In [ ]:
batch_tok_tfm = LMBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, lm_strategy_cls=BertMLMStrategy)

blocks = (TextBlock(batch_tokenize_tfm=batch_tok_tfm, input_return_type=MLMTextInput), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader(0), splitter=ColSplitter(col="is_valid"))


##### Step 3: Build your `DataLoaders`

In [ ]:
dls = dblock.dataloaders(df, bs=4)


In [ ]:
b = dls.one_batch()
b[0]["input_ids"].shape, b[0]["labels"].shape, b[1].shape


(torch.Size([4, 512]), torch.Size([4, 512]), torch.Size([4, 512]))

In [ ]:
b[0]["input_ids"][0][:20], b[0]["labels"][0][:20], b[1][0][:20]


(tensor([  101,   103,  3960,  7758,  1027,  3960,  7758,  1006,  1013,   103,
          4895,  2243,  1028,   103,  1025,  2141,   103,  5297, 27946,  1010],
        device='cuda:1'),
 tensor([-100, 1027, -100, -100, -100, 3960, -100, -100, -100, 1026, -100, -100,
         -100, 1013, -100, -100, 2728, -100, -100, -100], device='cuda:1'),
 tensor([-100, 1027, -100, -100, -100, 3960, -100, -100, -100, 1026, -100, -100,
         -100, 1013, -100, -100, 2728, -100, -100, -100], device='cuda:1'))

In [ ]:
explode_types(b)


{tuple: [dict, torch.Tensor]}

In [ ]:
# export
@typedispatch
def show_batch(
    # This typedispatched `show_batch` will be called for `MLMTextInput` typed inputs
    x: MLMTextInput,
    # Your targets
    y,
    # Your raw inputs/targets
    samples,
    # Your `DataLoaders`. This is required so as to get at the Hugging Face objects for
    # decoding them into something understandable
    dataloaders,
    # Your `show_batch` context
    ctxs=None,
    # The maximum number of items to show
    max_n=6,
    # Any truncation your want applied to your decoded inputs
    trunc_at=None,
    # Any other keyword arguments you want applied to `show_batch`
    **kwargs,
):
    # grab our tokenizer and ignore token to decode
    tfm = first_blurr_tfm(dataloaders)
    hf_tokenizer = tfm.hf_tokenizer
    ignore_token_id = tfm.ignore_token_id

    # grab our mask token id and do-not-mask token ids
    mask_token_id = hf_tokenizer.mask_token_id

    vocab = hf_tokenizer.get_vocab()
    dnm_tok_ids = [vocab[tok] for tok in list(hf_tokenizer.special_tokens_map.values()) if vocab[tok] != mask_token_id]

    res = L()
    for s in samples:
        # exclue dnm tokens from input
        inps = [
            hf_tokenizer.decode(tok_id) if (tok_id == mask_token_id or s[1][idx] == ignore_token_id) else f"[{hf_tokenizer.decode(tok_id)}]"
            for idx, tok_id in enumerate(s[0])
            if (tok_id not in dnm_tok_ids)
        ]

        # replaced masked tokens with "[{actual_token}]"
        trgs = [
            hf_tokenizer.decode(s[0][idx]) if (tok_id == ignore_token_id) else f"[{hf_tokenizer.decode(tok_id)}]"
            for idx, tok_id in enumerate(s[1])
            if (s[0][idx] not in dnm_tok_ids)
        ]

        res.append((" ".join(inps[:trunc_at]).strip(), " ".join(trgs[:trunc_at]).strip()))

    display_df(pd.DataFrame(res, columns=["text", "target"])[:max_n])
    return ctxs


In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=250)


,text,target
0,"= bob dylan = [MASK] dylan ( / < un [MASK] > / [baton] born robert allen zimmerman , may 24 , 1941 [)] is an american singer @ - @ [MASK] , artist and [##igny] . he has been influential in popular music [MASK] culture for more than [MASK] decades [MASK] much of his most celebrated work dates from the 1960s when his [MASK] chronicle ##d social unrest , although dylan rep ##udi ##ated suggestions from [MASK] that he was a spokesman for his [MASK] . nevertheless , early songs such as "" blow ##in ' [MASK] the wind "" and "" the [MASK] they are a @ - [MASK] < un ##k > ' [MASK] became anthem ##s for the american civil rights and anti @ - @ war movements . after [MASK] left his initial base [MASK] the american folk music revival , his six @ - [MASK] minute single "" like a rolling stone "" [MASK] the range [MASK] popular music in 1965 . [MASK] [MASK] @ - @ 1960s recordings , backed [MASK] rock musicians , reached the top end [MASK] [rpg] united states music charts [MASK] also attracting < un ##k > and criticism from others [MASK] the folk movement . dylan ' s lyrics have incorporated various political , social , [MASK] [MASK] [MASK] [MASK] influences . [MASK] def ##ied existing pop music conventions and appealed to the bu ##rgeon ##ing counter ##culture . initially inspired by the performances of little richard and","= bob dylan = [bob] dylan ( / < un [##k] > / [;] born robert allen zimmerman , may 24 , 1941 [)] is an american singer @ - @ [songwriter] , artist and [writer] . he has been influential in popular music [and] culture for more than [five] decades [.] much of his most celebrated work dates from the 1960s when his [songs] chronicle ##d social unrest , although dylan rep ##udi ##ated suggestions from [journalists] that he was a spokesman for his [generation] . nevertheless , early songs such as "" blow ##in ' [in] the wind "" and "" the [times] they are a @ - [@] < un ##k > ' [""] became anthem ##s for the american civil rights and anti @ - @ war movements . after [he] left his initial base [in] the american folk music revival , his six @ - [@] minute single "" like a rolling stone "" [altered] the range [of] popular music in 1965 . [his] [mid] @ - @ 1960s recordings , backed [by] rock musicians , reached the top end [of] [the] united states music charts [while] also attracting < un ##k > and criticism from others [in] the folk movement . dylan ' s lyrics have incorporated various political , social , [philosophical] [,] [and] [literary] influences . [they] def ##ied existing pop music conventions and appealed to the bu ##rgeon ##ing counter ##culture . initially inspired by the performances of little richard and"
1,"[MASK] [MASK] of romani = the battle of [MASK] was the last ground attack of the central powers on the suez [canal] at the beginning [MASK] the sinai and palestine campaign during the [MASK] world war . the [MASK] was fought between 3 and 5 august 1916 near [MASK] [MASK] [MASK] of romani and the [MASK] of [ancient] pe ##lus ##ium on the sinai [MASK] , 23 miles [MASK] 37 [MASK] ) east of the suez canal . [MASK] victory by the 52nd [MASK] lowland ) division and the anzac mounted division of [MASK] egyptian expeditionary force ( ee ##f ) over a joint [emergence] and german [MASK] , which had marched across the sinai , marked the end of [MASK] defence of the [MASK] [MASK] campaign , also [MASK] as the [MASK] zur < un ##k > [MASK] < un ##k > and the < [un] ##k > < un ##k > < un ##k > , which had begun on 26 [MASK] [MASK] . this british empire [MASK] [,] the first against the ottoman [MASK] in the war [MASK] ensured the safety [of] [MASK] suez canal from ground attacks , and [MASK] the central powers [MASK] ambitions [vicinity] [MASK] [differing] traffic through the canal by gaining control of the strategically important northern [MASK] to [MASK] suez canal . the pursuit [by] the [MASK] [mounted] division which ended [MASK] bi ##r el [MASK] on 12 august began the sinai and palestine campaign . thereaf

## Summary

You can implement whatever LM strategy your heart desires for either causal or masked language modeling tasks in Blurr

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()


Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted 99e_examples-causal-lm-gpt2.ipynb.
Converted index.ipynb.
